# Multiclass logistic regression with ``gluon``

Now that we've built a [logistic regression model from scratch](./P02-C03-softmax-regression-scratch.ipynb), let's make this more efficient with ``gluon``. If you completed the corresponding chapters on linear regression, you might be tempted rest your eyes a little in this one. We'll be using ``gluon`` in a rather similar way and since library is reasonably well designed, you won't have to do much differently. To keep you awake we'll introduce a few subtle tricks. 

Let's start by importing the standard packages. 

In [14]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

## Set the context

We'll set the cotext. In the linear regression tutorial we did all of our computation on the cpu (`mx.cpu()`) just to keep things simple. When you've got 2-dimensional data and scalar labels, a smartwatch can probably handle the job. Already, in this tutorial we'll be working with a considerably larger dataset. If you happen to be running this code on a server with a GPU and remembered to build MXNet with ``CUDA=1``, you might want to substitute the following line for its commented-out counterpart.

In [12]:
########################
#  Set the context to CPU
########################
ctx = mx.cpu()

########################
#  If you have GPU, instead call:
########################
#ctx = mx.gpu()

## The MNIST Dataset

We won't suck up too much wind describing the MNIST dataset for a second time. If you're unfamiliar with the dataset and are reading these chapters out of sequence, take a look at the data section in the previous chapter on [softmax regression from scratch](./P02-C03-softmax-regression-scratch.ipynb).

In [13]:
mnist = mx.test_utils.get_mnist()

## Data Iterators

We'll load up data iterators corresponding to the training and test splits of our dataset. 

In [4]:
batch_size = 64
train_data = mx.io.NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(mnist["test_data"], mnist["test_label"], batch_size, shuffle=True)

We're also going to want to load up an iterator with *test* data. After we train on the training dataset we're going to want to test our model on the test data. Otherwise, for all we know, our model could be doing something stupid (or treacherous?) like memorizing the training examples and regurgitating the labels on command.

## Multiclass Logistic Regression

Now we're going to define our model. Remember that before we added ``Dense`` layers by calling ``net.add(gluon.nn.Dense(num_outputs, in_units=num_inputs))``. 
This tells ``gluon`` all that it needs in order allocate memory for the weights.
After that, all we need to do is initialize the weights, instantiate a loss and an optimzer, and we can start training. The most straightforward way to do this is to call the following code:

In [15]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(10, in_units=784))

## Shape inference

One slick feature that we can take advantage of in ``gluon`` is shape inference on parameters. 
Instead of explicitly declaring the number of inputs to a layer, 
we can simply state the number of outputs. 

In [16]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(10))

You might wonder, how can gluon allocate our parameters if it doesn't know what shape they should take? We'll elaborate on this and more of ``gluon``'s internal workings in [our chapter on plumbing](./P03.5-C01-plumbing.ipynb), but here's the short version. In fact, ``gluon`` doesn't allocate our parameters. Instead it defers allocation and initialization to the first time we actually make a forward pass through the model with real data. Then, when ``gluon`` sees the shape of our data, it can infer the shapes of all of the parameters and perform the deferred initialization.

## Parameter initialization


In [17]:
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=ctx)

## Softmax Cross Entropy Loss

Note, we didn't have to include the softmax layer because MXNet's has an efficient function that simultaneously computes the softmax activation and cross-entropy loss. However, if ever need to get the output probabilities, 

In [18]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

And let's instantiate an optimizer to make our updates

In [19]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

## Evaluation Metric

This time, let's simplify the evaluation code by relying on MXNet's built-in ``metric`` package.

In [20]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx).reshape((-1,784))
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

Because we initialized our model randomly, and because roughly one tenth of all examples belong to each of the ten classes, we should have an accuracy in the ball park of .10.

In [21]:
evaluate_accuracy(test_data, net)

0.063694267515923567

## Execute training loop

In [22]:
epochs = 10
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx).reshape((-1,784))
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
        cross_entropy.backward()
        trainer.step(data.shape[0])
        
        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = np.mean(cross_entropy.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(cross_entropy.asnumpy()[0])
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))    
    

Epoch 0. Loss: 1.20148020521, Train_acc 0.792543976546, Test_acc 0.798268312102
Epoch 1. Loss: 0.816217981854, Train_acc 0.837203491471, Test_acc 0.842157643312
Epoch 2. Loss: 0.673927094833, Train_acc 0.855293843284, Test_acc 0.859574044586
Epoch 3. Loss: 0.590060518551, Train_acc 0.865021988273, Test_acc 0.870123407643
Epoch 4. Loss: 0.530897087987, Train_acc 0.872601279318, Test_acc 0.875895700637
Epoch 5. Loss: 0.487312493866, Train_acc 0.8780483742, Test_acc 0.882066082803
Epoch 6. Loss: 0.454868255366, Train_acc 0.882512659915, Test_acc 0.884952229299
Epoch 7. Loss: 0.43002824538, Train_acc 0.885744269723, Test_acc 0.886843152866
Epoch 8. Loss: 0.410234067782, Train_acc 0.888542777186, Test_acc 0.889331210191
Epoch 9. Loss: 0.393877197405, Train_acc 0.891074760128, Test_acc 0.891321656051


## Conclusion

Now let's take a look at how to implement modern neural networks. 

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)